In [ ]:
#copy ranking data paste into Google Sheets
#split text into columns
#save as csv
#upload into Google Sheets
#split text into columns
#take of crosses for dead
!pip install fuzzywuzzy
!pip install python-Levenshtein

In [ ]:
import pandas as pd
import numpy as np
import mysql.connector as sql
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

In [ ]:
rankings_data = pd.read_csv("/Users/amedeusdsouza/Desktop/ura-mike/rankings_top_five_pct_aug_2020_cleaned.csv")

In [ ]:
rankings_data

In [ ]:
db_connection = sql.connect(host="_____", database="_____", user="_____", password="_____")

db_cursor = db_connection.cursor()

db_cursor.execute('select * from recommenders')

table_rows = db_cursor.fetchall()

mike_data = pd.DataFrame(table_rows)

In [ ]:
mike_data = mike_data.rename(columns = {0: "sid",
                                       1: "firstname",
                                       2: "lastname",
                                       3: "email",
                                       4: "orgid",
                                       5: "inst_name",
                                       6: "inst_shortname",
                                       7: "deptname"})

In [ ]:
mike_data

In [ ]:
rankings_data = rankings_data.applymap(lambda i:i.lower() if type(i) == str else i) #lowercase strings
mike_data = mike_data.applymap(lambda i:i.lower() if type(i) == str else i)

In [ ]:
rankings_data

In [ ]:
mike_data["name"] = mike_data["firstname"] + " " + mike_data["lastname"]

In [ ]:
merge_1 = pd.merge(left = rankings_data, right = mike_data, on = "name", how = "left")

In [ ]:
merge_1.columns

In [ ]:
merge_1[merge_1.sid.notnull()]

In [ ]:
merge_1[merge_1.name == "thomas lemieux"] #it works

In [ ]:
rankings_data[rankings_data.name == "michael peters"] 

In [ ]:
merge_1[merge_1.name == "michael peters"] #multiple mikes

In [ ]:
merge_1["1a_in_inst_name"] = merge_1[merge_1.inst1a.notnull() & merge_1.inst_name.notnull()].apply(lambda x: x.inst1a in x.inst_name, axis = 1)
merge_1["1b_in_inst_name"] = merge_1[merge_1.inst1b.notnull() & merge_1.inst_name.notnull()].apply(lambda x: x.inst1b in x.inst_name, axis = 1)
merge_1["1c_in_inst_name"] = merge_1[merge_1.inst1c.notnull() & merge_1.inst_name.notnull()].apply(lambda x: x.inst1c in x.inst_name, axis = 1)
merge_1["1d_in_inst_name"] = merge_1[merge_1.inst1d.notnull() & merge_1.inst_name.notnull()].apply(lambda x: x.inst1d in x.inst_name, axis = 1)
merge_1["1e_in_inst_name"] = merge_1[merge_1.inst1e.notnull() & merge_1.inst_name.notnull()].apply(lambda x: x.inst1e in x.inst_name, axis = 1)
merge_1["1f_in_inst_name"] = merge_1[merge_1.inst1f.notnull() & merge_1.inst_name.notnull()].apply(lambda x: x.inst1f in x.inst_name, axis = 1)
merge_1["2a_in_inst_name"] = merge_1[merge_1.inst2a.notnull() & merge_1.inst_name.notnull()].apply(lambda x: x.inst2a in x.inst_name, axis = 1)
merge_1["2b_in_inst_name"] = merge_1[merge_1.inst2b.notnull() & merge_1.inst_name.notnull()].apply(lambda x: x.inst2b in x.inst_name, axis = 1)
merge_1["2c_in_inst_name"] = merge_1[merge_1.inst2c.notnull() & merge_1.inst_name.notnull()].apply(lambda x: x.inst2c in x.inst_name, axis = 1)
merge_1["2d_in_inst_name"] = merge_1[merge_1.inst2d.notnull() & merge_1.inst_name.notnull()].apply(lambda x: x.inst2d in x.inst_name, axis = 1)
merge_1["2e_in_inst_name"] = merge_1[merge_1.inst2e.notnull() & merge_1.inst_name.notnull()].apply(lambda x: x.inst2e in x.inst_name, axis = 1)
#merge_1["2f_in_inst_name"] = merge_1[merge_1.inst2f.notnull() & merge_1.inst_name.notnull()].apply(lambda x: x.inst2f in x.inst_name, axis = 1)
merge_1["2f_in_inst_name"] = merge_1.inst2f.notnull()

In [ ]:
merge_1.columns

In [ ]:
merge_1["good_match"] = merge_1["1a_in_inst_name"]|merge_1["1b_in_inst_name"]|merge_1["1c_in_inst_name"]|merge_1["1d_in_inst_name"]|merge_1["1e_in_inst_name"]|merge_1["1f_in_inst_name"]|merge_1["2a_in_inst_name"]|merge_1["2b_in_inst_name"]|merge_1["2c_in_inst_name"]|merge_1["2d_in_inst_name"]|merge_1["2e_in_inst_name"]|merge_1["2f_in_inst_name"]


In [ ]:
merge_1_good = merge_1[merge_1["good_match"] == True]

In [ ]:
merge_1_good

In [ ]:
merge_1_good[merge_1_good.name == "michael peters"]

In [ ]:
rankings_to_merge_post_level_1 =  rankings_data[~rankings_data.repec_rank.isin(list(merge_1_good.repec_rank))]

In [ ]:
rankings_to_merge_post_level_1

In [ ]:
mike_names_list = mike_data["name"].tolist() #list of supervisor names

In [ ]:
a = rankings_to_merge_post_level_1["name"].head() #test to see if process extract works with slice

In [ ]:
b = mike_names_list[0:25] #test to see if process extract works with slice

In [ ]:
a.apply(lambda x :  process.extract(x, b, limit = 1)) #as expected

In [ ]:
a #as expected compare with above

In [ ]:
all(isinstance(i, str) for i in mike_names_list) #all elements of list are strings

In [ ]:
rankings_process = rankings_to_merge_post_level_1["name"].apply(lambda x : process.extract(x, mike_names_list, limit = 1)) 
#series of names matched with ranking data institution; format [(name, levenshtein score)]

In [ ]:
#rankings_to_merge_post_level_1["supname_process"] = rankings_process #create column using series

In [ ]:
#rankings_to_merge_post_level_1["supname_process"] = pd.DataFrame(rankings_to_merge_post_level_1.supname_process.tolist(), index = rankings_to_merge_post_level_1.index) #0th element of list

In [ ]:
#rankings_data[["name", "match_score"]] = pd.DataFrame(rankings_data.supname.tolist(), index = rankings_data.index) #split tuple into elements in two columns

In [ ]:
#rankings_data_perfect_match = rankings_data[rankings_data["match_score"] == 100]
#rankings_data_imperfect_match = rankings_data[rankings_data["match_score"] != 100]